In [2]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np 
import numpy as np
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import scipy.linalg as slin
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import glob
import re
import math
from torch.optim.adam import Adam
from utils import *
from statistics import mean
import random


Bad key "text.kerning_factor" on line 4 in
D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
np.random.seed(1234567)
def simulate_random_dag(d: int,
                        degree: float,
                        w_range: tuple = (1.0, 1.0)) -> nx.DiGraph:
    """Simulate random DAG with an expected degree by Erdos-Renyi model.
        
        Args:
        d: number of nodes
        degree: expected node degree, in + out
        w_range: weight range +/- (low, high)
        
        Returns:
        G: weighted DAG
        """
    prob = float(degree) / (d - 1)
    B = np.tril((np.random.rand(d, d) < prob).astype(float), k=-1)
    
    # random permutation
    P = np.random.permutation(np.eye(d, d))  # permutes first axis only
    B_perm = P.T.dot(B).dot(P)
    U = np.random.uniform(low=w_range[0], high=w_range[1], size=[d, d])
    U[np.random.rand(d, d) < 0.5] *= -1
    W = (B_perm != 0).astype(float) * U
    
    # remove all in-edges (from precedent nodes) of the first node as A
    W[:, 0] = 0
    # remove all out-edges (from descendent nodes) of the last node as Y
    W[d-1, :] = 0
    # the remained nodes are the mediators M; and reset mediators if it has higher topological order than A or lower order than Y.
    ordered_vertices = list(nx.topological_sort(nx.DiGraph(W)))
    j = 1
    while j < d - 1:
        if  ordered_vertices.index(j) < ordered_vertices.index(0):
            W[j, 1:(d - 1)] = np.zeros (d - 2)
        if  ordered_vertices.index(j) > ordered_vertices.index(d - 1):
            W[1:(d - 1), j] = np.zeros (d - 2)
        j = j + 1
    #print("True weighted adjacency matrix B:\n", W)
    G = nx.DiGraph(W)
    calculate_effect(W)
    return W
import math 
def cos(x):
    return math.cos(x/4*math.pi)*0.8

In [4]:
def data_create(seed,time_stamp):
  np.random.seed(seed)
  x = np.array(range(0, time_stamp))
  #y = np.array([cos(i)+np.random.normal(0, 0.1, 1) for i in x]) #generate coefficient
  y = np.array([cos(i)for i in x]) #generate coefficient
  base_DAG=np.zeros((5, 5))
  base_DAG[0,4]=y[0]##edited to be coeffcient with error
  base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
  X_all = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
  for i in range(1,time_stamp):
      base_DAG[0,4]=y[i]
      base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
      X = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
      X_all=np.append(X_all,X,axis=0)
  return X_all

In [9]:
n_var=5
n_times=30 #no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list_new=np.zeros((n_times,time_stamp,n_var,n_var))
lambda_list=np.zeros((n_times,30,n_var))
seed=seed_list[0]
X_all=data_create(seed,time_stamp) #create data

In [10]:
X_all.shape

(300, 5, 1)

In [1]:
def notears_linear(X, lambda1, loss_type, max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3):
    """Solve min_W L(W; X) + lambda1 ‖W‖_1 s.t. h(W) = 0 using augmented Lagrangian.

    Args:
        X (np.ndarray): [n, d] sample matrix
        lambda1 (float): l1 penalty parameter
        loss_type (str): l2, logistic, poisson
        max_iter (int): max num of dual ascent steps
        h_tol (float): exit if |h(w_est)| <= htol
        rho_max (float): exit if rho >= rho_max
        w_threshold (float): drop edge if |weight| < threshold

    Returns:
        W_est (np.ndarray): [d, d] estimated DAG
    """
    def _loss(W):
        """Evaluate value and gradient of loss."""
        M = X @ W
        if loss_type == 'l2':
            R = X - M
            loss = 0.5 / X.shape[0] * (R ** 2).sum()
            G_loss = - 1.0 / X.shape[0] * X.T @ R
        elif loss_type == 'logistic':
            loss = 1.0 / X.shape[0] * (np.logaddexp(0, M) - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (sigmoid(M) - X)
        elif loss_type == 'poisson':
            S = np.exp(M)
            loss = 1.0 / X.shape[0] * (S - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (S - X)
        else:
            raise ValueError('unknown loss type')
        return loss, G_loss

    def _h(W):
        """Evaluate value and gradient of acyclicity constraint."""
        E = slin.expm(W * W)  # (Zheng et al. 2018)
        h = np.trace(E) - d
        #     # A different formulation, slightly faster at the cost of numerical stability
        #     M = np.eye(d) + W * W / d  # (Yu et al. 2019)
        #     E = np.linalg.matrix_power(M, d - 1)
        #     h = (E.T * M).sum() - d
        G_h = E.T * W * 2
        return h, G_h

    def _adj(w):
        """Convert doubled variables ([2 d^2] array) back to original variables ([d, d] matrix)."""
        return (w[:d * d] - w[d * d:]).reshape([d, d])

    def _func(w):
        """Evaluate value and gradient of augmented Lagrangian for doubled variables ([2 d^2] array)."""
        W = _adj(w)
        loss, G_loss = _loss(W)
        h, G_h = _h(W)
        obj = loss + 0.5 * rho * h * h + alpha * h + lambda1 * w.sum()
        G_smooth = G_loss + (rho * h + alpha) * G_h
        g_obj = np.concatenate((G_smooth + lambda1, - G_smooth + lambda1), axis=None)
        return obj, g_obj

    n, d = X.shape
    w_est, rho, alpha, h = np.zeros(2 * d * d), 1.0, 0.0, np.inf  # double w_est into (w_pos, w_neg)
    bnds = [(0, 0) if i == j else (0, None) for _ in range(2) for i in range(d) for j in range(d)]
    if loss_type == 'l2':
        X = X - np.mean(X, axis=0, keepdims=True)
    for _ in range(max_iter):
        w_new, h_new = None, None
        while rho < rho_max:
            sol = sopt.minimize(_func, w_est, method='L-BFGS-B', jac=True, bounds=bnds)
            w_new = sol.x
            h_new, _ = _h(_adj(w_new))
            if h_new > 0.25 * h:
                rho *= 10
            else:
                break
        w_est, h = w_new, h_new
        alpha += rho * h
        if h <= h_tol or rho >= rho_max:
            break
    W_est = _adj(w_est)
    W_est[np.abs(W_est) < w_threshold] = 0
    return W_est

In [22]:
X_all.reshape(300,5)[:30].shape

(30, 5)

In [33]:
import scipy.optimize as sopt
n_times=30#no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list_new=np.zeros((n_times,time_stamp,n_var,n_var))

for replicate in range(n_times):
  seed=seed_list[replicate]
  X_all=data_create(seed,time_stamp) #create data
  average_list=np.zeros((time_stamp,n_var, n_var))
  for time in range(time_stamp):
    X=X_all.reshape(300,5)[30*time:(30*time+30)]
    average_list[time,:,:]=notears_linear(X, lambda1=0.1, loss_type="l2", max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3)
    average_coef_list_new[replicate,:,:,:]=average_list

In [34]:
np.save("cos_10_30_NOTEARS",average_coef_list_new)

In [35]:
import math 
def cos(x):
    return(-10+(5-x)**2)/20

In [36]:
def data_create(seed,time_stamp):
  np.random.seed(seed)
  x = np.array(range(0, time_stamp))
  #y = np.array([cos(i)+np.random.normal(0, 0.1, 1) for i in x]) #generate coefficient
  y = np.array([cos(i)for i in x]) #generate coefficient
  base_DAG=np.zeros((5, 5))
  base_DAG[0,4]=y[0]##edited to be coeffcient with error
  base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
  X_all = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
  for i in range(1,time_stamp):
      base_DAG[0,4]=y[i]
      base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
      X = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
      X_all=np.append(X_all,X,axis=0)
  return X_all

In [37]:
import scipy.optimize as sopt
n_times=30#no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list_new=np.zeros((n_times,time_stamp,n_var,n_var))

for replicate in range(n_times):
  seed=seed_list[replicate]
  X_all=data_create(seed,time_stamp) #create data
  average_list=np.zeros((time_stamp,n_var, n_var))
  for time in range(time_stamp):
    X=X_all.reshape(300,5)[30*time:(30*time+30)]
    average_list[time,:,:]=notears_linear(X, lambda1=0.1, loss_type="l2", max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3)
    average_coef_list_new[replicate,:,:,:]=average_list
np.save("quadra_10_30_NOTEARS",average_coef_list_new)

In [38]:
def simulate_random_dag(d: int,
                        degree: float,
                        w_range: tuple = (1.0, 1.0)) -> nx.DiGraph:
    """Simulate random DAG with an expected degree by Erdos-Renyi model.
        
        Args:
        d: number of nodes
        degree: expected node degree, in + out
        w_range: weight range +/- (low, high)
        
        Returns:
        G: weighted DAG
        """
    prob = float(degree) / (d - 1)
    B = np.tril((np.random.rand(d, d) < prob).astype(float), k=-1)
    
    # random permutation
    P = np.random.permutation(np.eye(d, d))  # permutes first axis only
    B_perm = P.T.dot(B).dot(P)
    U = np.random.uniform(low=w_range[0], high=w_range[1], size=[d, d])
    U[np.random.rand(d, d) < 0.5] *= -1
    W = (B_perm != 0).astype(float) * U
    
    # remove all in-edges (from precedent nodes) of the first node as A
    W[:, 0] = 0
    # remove all out-edges (from descendent nodes) of the last node as Y
    W[d-1, :] = 0
    # the remained nodes are the mediators M; and reset mediators if it has higher topological order than A or lower order than Y.
    ordered_vertices = list(nx.topological_sort(nx.DiGraph(W)))
    j = 1
    while j < d - 1:
        if  ordered_vertices.index(j) < ordered_vertices.index(0):
            W[j, 1:(d - 1)] = np.zeros (d - 2)
        if  ordered_vertices.index(j) > ordered_vertices.index(d - 1):
            W[1:(d - 1), j] = np.zeros (d - 2)
        j = j + 1
    #print("True weighted adjacency matrix B:\n", W)
    G = nx.DiGraph(W)
    calculate_effect(W)
    return W

In [39]:
np.random.seed(123456)
base_DAG=simulate_random_dag(5,4)
base_DAG
random.seed(1234567)
pick_element=random.sample(range(5), 2) ## pick two location to mutate
pick_1=np.nonzero(base_DAG)[0][pick_element[0]],np.nonzero(base_DAG)[1][pick_element[0]]
pick_2=np.nonzero(base_DAG)[0][pick_element[1]],np.nonzero(base_DAG)[1][pick_element[1]]

The total effect (TE): 0.0
The natural direct effect (DE): 0.0
The natural indirect effect (IE): 0.0
The natural direct effect for mediators (DM): [ 0.  0. -0.]
The natural direct effect for mediators (IM): [0. 0. 0.]


In [40]:
import math 
def cos(x):
    return ((math.cos(x/6*math.pi))/4+1)*0.8
def quadratic(x):
    return(-5+(10-x)**2)/200+0.5

In [41]:
def data_create(seed,time_stamp,set_graph):
  np.random.seed(seed)
  x = np.array(range(0, time_stamp))
  #y = np.array([cos(i)+np.random.normal(0, 0.1, 1) for i in x]) #generate coefficient
  y=np.array([cos(i) for i in x])
  z=np.array([quadratic(i) for i in x])##edited to have multiple
  base_DAG=set_graph 
  base_DAG[pick_1]=y[0]##edited to be coeffcient with error
  base_DAG[pick_2]=z[0]###multiple
  base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
  X_all = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
  for i in range(1,time_stamp):
      base_DAG[pick_1]=y[i]##edited to be coeffcient with error
      base_DAG[pick_2]=z[i]###multiple
      base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
      X = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
      X_all=np.append(X_all,X,axis=0)
      print(base_DAG)
  return X_all

In [43]:
import scipy.optimize as sopt
n_times=30#no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list_new=np.zeros((n_times,time_stamp,n_var,n_var))

for replicate in range(n_times):
  seed=seed_list[replicate]
  X_all=data_create(seed,time_stamp,base_DAG) #create data
  average_list=np.zeros((time_stamp,n_var, n_var))
  for time in range(time_stamp):
    X=X_all.reshape(300,5)[30*time:(30*time+30)]
    average_list[time,:,:]=notears_linear(X, lambda1=0.1, loss_type="l2", max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3)
    average_coef_list_new[replicate,:,:,:]=average_list
np.save("full_10_30_NOTEARS",average_coef_list_new)

[[ 0.         -1.         -0.          0.88        0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.          1.         -0.          0.          0.97320508]
 [ 0.         -0.          0.          0.         -0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[ 0.    -1.    -0.     0.795  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.9  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.   -1.   -0.    0.72  0.  ]
 [ 0.    0.    0.    0.   -0.  ]
 [ 0.    1.   -0.    0.    0.8 ]
 [ 0.   -0.    0.    0.   -0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.    -1.    -0.     0.655  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.7  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.         -1.         -0.          0.6         0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.     

[[ 0.         -1.         -0.          0.88        0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.          1.         -0.          0.          0.97320508]
 [ 0.         -0.          0.          0.         -0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[ 0.    -1.    -0.     0.795  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.9  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.   -1.   -0.    0.72  0.  ]
 [ 0.    0.    0.    0.   -0.  ]
 [ 0.    1.   -0.    0.    0.8 ]
 [ 0.   -0.    0.    0.   -0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.    -1.    -0.     0.655  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.7  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.         -1.         -0.          0.6         0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.     

[[ 0.         -1.         -0.          0.88        0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.          1.         -0.          0.          0.97320508]
 [ 0.         -0.          0.          0.         -0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[ 0.    -1.    -0.     0.795  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.9  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.   -1.   -0.    0.72  0.  ]
 [ 0.    0.    0.    0.   -0.  ]
 [ 0.    1.   -0.    0.    0.8 ]
 [ 0.   -0.    0.    0.   -0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.    -1.    -0.     0.655  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.7  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.         -1.         -0.          0.6         0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.     

[[ 0.         -1.         -0.          0.88        0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.          1.         -0.          0.          0.97320508]
 [ 0.         -0.          0.          0.         -0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[ 0.    -1.    -0.     0.795  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.9  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.   -1.   -0.    0.72  0.  ]
 [ 0.    0.    0.    0.   -0.  ]
 [ 0.    1.   -0.    0.    0.8 ]
 [ 0.   -0.    0.    0.   -0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.    -1.    -0.     0.655  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.7  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.         -1.         -0.          0.6         0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.     

[[ 0.         -1.         -0.          0.88        0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.          1.         -0.          0.          0.97320508]
 [ 0.         -0.          0.          0.         -0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[ 0.    -1.    -0.     0.795  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.9  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.   -1.   -0.    0.72  0.  ]
 [ 0.    0.    0.    0.   -0.  ]
 [ 0.    1.   -0.    0.    0.8 ]
 [ 0.   -0.    0.    0.   -0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.    -1.    -0.     0.655  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.7  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.         -1.         -0.          0.6         0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.     

[[ 0.         -1.         -0.          0.88        0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.          1.         -0.          0.          0.97320508]
 [ 0.         -0.          0.          0.         -0.        ]
 [ 0.          0.          0.          0.          0.        ]]
[[ 0.    -1.    -0.     0.795  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.9  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.   -1.   -0.    0.72  0.  ]
 [ 0.    0.    0.    0.   -0.  ]
 [ 0.    1.   -0.    0.    0.8 ]
 [ 0.   -0.    0.    0.   -0.  ]
 [ 0.    0.    0.    0.    0.  ]]
[[ 0.    -1.    -0.     0.655  0.   ]
 [ 0.     0.     0.     0.    -0.   ]
 [ 0.     1.    -0.     0.     0.7  ]
 [ 0.    -0.     0.     0.    -0.   ]
 [ 0.     0.     0.     0.     0.   ]]
[[ 0.         -1.         -0.          0.6         0.        ]
 [ 0.          0.          0.          0.         -0.        ]
 [ 0.     

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in multiply
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in double_scalars
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in multiply
